In [1]:
using Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate();

  Activating project at `~/Desktop/Work/Research/LEGO-project/LEGO-3D_Dyn`


In [2]:
import ForwardDiff as FD
using LinearAlgebra
using Plots

In [22]:
function newton_step(func::Function, x0::Vector)::Vector
	# function value at current point
  f = func(x0)
  # check if function is scalar or vector and calculate the Jacobian accordingly
  if length(f) == 1
    J = FD.gradient(func, x0)
    return x0 .- f[1] / J[1]
  else
    J = FD.jacobian(func, x0)
    return x0 - J \ f
  end 
end

function newton_root_finding(func::Function, x0::Vector; tol=1e-6, max_iter=100)::Vector
  x = x0
  for i in 1:max_iter
		# take newton step
    x_new = newton_step(func, x)
    if func(x_new) < tol
      
      return x_new
    end
    x = x_new
  end
	throw("Max iterations reached")
  return x
end

newton_root_finding (generic function with 1 method)